In [4]:
# Widgets
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import Layout, interact

# Imports
import pandas as pd
import numpy as np
import math

import io
import requests

import time

import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = "https://spreadsheets.google.com/feeds"
credentials = ServiceAccountCredentials.from_json_keyfile_name("matcher-272116-8801bce55bcb.json", scope)
gs = gspread.authorize(credentials)

data = None

# Use journey data from express guaranteed Google Sheet?

In [5]:
express_guaranteed = widgets.Dropdown(
    options = ["Select option", "Cyfe Board (May be empty now)", "Express Guaranteed Sheet"],
    value = "Select option",
    description = "Where to pull journey data from?",
    layout = Layout(width = "50%"),
    style = {"description_width":"initial"})

express_guaranteed_button = widgets.Button(description = "Load", layout = Layout(width = "80px"))
out = widgets.Output()

def load_journey_data(_) :
    global data, continue_load
    with out :
        clear_output()
        print("Loading journeys", end = "\r")
        
        if express_guaranteed.value == "Express Guaranteed Sheet" :
            url = "https://docs.google.com/spreadsheets/d/1o7jEklbPYk-7So0UALnt8dZCYlIbdyXhzsgm9MHf82I/edit#gid=0"

            express_guaranteed_sheet = gs.open_by_url(url).worksheet("journeys").get_all_values()
            data = pd.DataFrame(express_guaranteed_sheet[1:], columns = express_guaranteed_sheet[0])
            data = data.replace("", np.nan)
            print("{0} journeys loaded    ".format(len(data.index)))
        elif express_guaranteed.value == "Cyfe Board (May be empty now)" :
            url = "https://www.anyvan.com/board/cyfe/express_interest_expresses"
            s = requests.get(url).content

            try :
                data = pd.read_csv(io.StringIO(s.decode("utf-8"))).sort_values(by = ["start_end"], ascending = False)
                print("{0} journeys loaded    ".format(len(data.index)))
            except :
                data = None
                print("No journeys available")
        else :
            data = None
            print("Select one of the options")
                
express_guaranteed_button.on_click(load_journey_data)

display(widgets.VBox([widgets.HBox([express_guaranteed, express_guaranteed_button]), out]))

In [6]:
data

,journey_id,tp_amount,name#reg,tp,tp_reg,dist_to_start,dist_to_end,min_dist,start_lat,start_lng,...,number_of_men,tail_lift,max_m3,max_kg,veh_lat,veh_lng,dist_combined,start_time,postcode,*https://www.anyvan.com/board/cyfe/express_interest_expresses
0,33607,265,NaN,NaN,NaN,NaN,NaN,NaN,51.72,0.59,...,1,1,7.71,446,NaN,NaN,NaN,09:00,CM3,NaN
1,33599,180,A1Van #FD61BLV,A1Van,FD61BLV,25.03,30.12,30.12,51.63,-0.19,...,1,0,6.81,551,51.55,0.15,55.14511158,10:00,N20,NaN
2,33609,211,A1Van #FD61BLV,A1Van,FD61BLV,25.48,19.57,19.57,51.42,-0.15,...,1,0,7.34,386,51.55,0.15,45.04760122,10:00,SW17,NaN
3,33619,181,A1Van #FD61BLV,A1Van,FD61BLV,27.94,16.05,16.05,51.47,-0.24,...,1,0,4.05,281,51.55,0.15,43.99661825,09:50,SW13,NaN
4,33625,219,A1Van #FD61BLV,A1Van,FD61BLV,19.90,21.23,21.23,51.66,-0.08,...,1,0,7.52,560,51.55,0.15,41.12935262,09:00,EN2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,33614,201,Zapoman #EO68CNN,Zapoman,EO68CNN,14.27,11.68,11.68,51.39,-0.26,...,1,0,7.16,441,51.51,-0.34,25.95157834,0.34375,KT3,NaN
353,33620,335,Zapoman #EO68CNN,Zapoman,EO68CNN,29.15,56.64,56.64,51.52,-0.76,...,2,0,7.38,364,51.51,-0.34,85.79159417,0.3590277778,SL6,NaN
354,33623,195,Zapoman #EO68CNN,Zapoman,EO68CNN,9.60,18.31,18.31,51.49,-0.20,...,1,0,6.28,382,51.51,-0.34,27.91000212,0.375,SW5,NaN
355,33625,219,Zapoman #EO68CNN,Zapoman,EO68CNN,24.54,20.93,20.93,51.66,-0.08,...,1,0,7.52,560,51.51,-0.34,45.46258575,0.375,EN2,NaN


# TPs

In [7]:
TPs = None

TPs_button = widgets.Button(description = "Load TPs", layout = Layout(width = "80 px"))
out = widgets.Output()

def load_TPs(_) :
    global TPs
    with out :
        clear_output()
        print("Loading TPs", end = "\r")
        
        # Returns
        url = "https://docs.google.com/spreadsheets/d/1_syJVv1Uni8oSa9BhtAjVcl2EZYZmJn03VKg9vofcD4/edit#gid=0"

        return_journey_converter_sheet = gs.open_by_url(url).worksheet("Input").get_all_values()[15:]
        returns = pd.DataFrame(return_journey_converter_sheet[1:], columns = return_journey_converter_sheet[0])
        returns = returns.drop(columns = returns.columns[16:])
        returns["nickname"].replace("", float("NaN"), inplace = True)
        returns = returns.dropna()

        # Providers
        sizes = {"Luton Van" : 20, 
                 "LWB up to 4m" : 10.5,
                 "MWB up to 3m" : 7.5,
                 "Not applicable" : 12,
                 "XLWB up 4m+" : 14.5,
                 "SWB up to 2.4m" : 5.5,
                 "Small Van" : 8}

        url = "https://docs.google.com/spreadsheets/d/1t0L9sgsUt-BT7wJWBNZ4fNY-Bo5GNytXP1wmkejFVw0/edit#gid=1205069010"

        TP_log_providers_sheet = gs.open_by_url(url).worksheet("format2").get_all_values()[7:]
        TP_log_providers = pd.DataFrame(TP_log_providers_sheet[1:], columns = TP_log_providers_sheet[0])
        providers = TP_log_providers.where(np.logical_and(TP_log_providers["vehicle"] != "N/A", np.logical_and(TP_log_providers["TP & REG"] != "AVStore #YN19FHD", np.logical_and(TP_log_providers["TP & REG"] != "1London #YN19FHF", TP_log_providers["TP & REG"] != "DWB91 #YN19FHD")))).dropna(how = "all").reset_index(drop = True).drop(columns = TP_log_providers.columns[-1])
        date = providers.columns[-1]

        providers["Size"] = [sizes[vehicle] for vehicle in providers["vehicle"]]
        providers["National?"] = [int(math.ceil(55 + (100 - float(nuts2c)) / 2 + (2000 if code.endswith("N") else 0))) for nuts2c,code in zip(providers["nuts2c"], providers[date])]
        providers["1Man"] = [1 if (code.startswith("1") or code.startswith("O")) else 0 for code in providers[date]]
        providers["2Man"] = [2 if (code.startswith("2") or code.startswith("O")) else 0 for code in providers[date]]
        providers["returns"] = [1 if TP in returns["name#reg"] else 0 for TP in providers["TP & REG"]]
        providers["guaranteed"] = [1 if isreturn == 1 else -1 for isreturn in providers["returns"]]
        providers["special code"] = ["2man-TL-LDN-cc" if (TwoMan == 2 and TL == "1") else "TL-LDN-cc" if (TL == "1") else "2man-LDN-cc" if (TwoMan == 2) else "LDN-cc" for TwoMan,TL in zip(providers["2Man"], providers["TL"])]
        providers["capacity"] = [16 if size >= 16 else size for size in providers["Size"]]

        providers = providers.sort_values(by = ["TP & REG"]).reset_index(drop = True)
        providers["index"] = providers.index

        # Expressing TPs
        columns = ["TP", "lat", "lng", "expressions", "guaranteed", "boooked"]
        expressing_TPs = pd.DataFrame(columns = columns)

        if data is not None :
            for TP in data["name#reg"].dropna().unique() :
                data_TP = data.where(data["name#reg"] == TP).dropna(how = "all")
                lat = round(float(data_TP.iloc[0]["veh_lat"]), 2)
                lng = round(float(data_TP.iloc[0]["veh_lng"]), 2)
                expressions = len(data_TP.index)
                guaranteed = 1 if (TP in providers["TP & REG"] or TP in returns["name#reg"]) else 0
                booked = 1 if TP in returns["name#reg"] else 0
                expressing_TPs = expressing_TPs.append(pd.DataFrame([[TP, lat, lng, expressions, guaranteed, booked]], columns = columns))
        else :
            print("No journeys available")

        expressing_TPs = expressing_TPs.sort_values(by = ["TP"]).reset_index(drop = True)
        expressing_TPs["index"] = expressing_TPs.index

        # Combine TPs
        TP_namereg = np.append(providers["TP & REG"], expressing_TPs["TP"])
        TP_lat = np.append(providers["Lat"], expressing_TPs["lat"])
        TP_lng = np.append(providers["Lng"], expressing_TPs["lng"])
        TP_TL = np.append(providers["TL"], np.full(len(expressing_TPs.index), np.nan))
        TP_capacity = np.append(providers["Size"], np.full(len(expressing_TPs.index), np.nan))
        TP_maxkm = np.append(providers["National?"], np.full(len(expressing_TPs.index), np.nan))
        TP_1man = np.append(providers["1Man"], np.full(len(expressing_TPs.index), np.nan))
        TP_2man = np.append(providers["2Man"], np.full(len(expressing_TPs.index), np.nan))
        TP_guaranteed = np.append(providers["guaranteed"], expressing_TPs["guaranteed"])
        TP_provider = np.append(np.ones(len(providers.index), dtype = bool), np.zeros(len(expressing_TPs.index), dtype = bool))

        columns = ["TP", "lat", "lng", "TL", "capacity", "maxkm", "1man", "2man", "guaranteed", "provider"]
        TPs = pd.DataFrame(zip(TP_namereg, TP_lat, TP_lng, TP_TL, TP_capacity, TP_maxkm, TP_1man, TP_2man, TP_guaranteed, TP_provider), columns = columns)
        TPs = TPs.where(TPs["guaranteed"] <= 0).dropna(how = "all").sort_values(by = ["provider", "TP"], ascending = [False, True])
        
        print("{0} TPs loaded      ".format(len(TPs.index)))
    
TPs_button.on_click(load_TPs)

widgets.VBox([TPs_button, out])

# Calculate iarray

In [8]:
iarray_button = widgets.Button(description = "Calculate IArray", layout = Layout(width = "150px"))

def distance_funct(start_lat, start_lng, lat, lng) :
    return 2 * 6371 * np.arcsin(np.sqrt(np.sin((start_lat - lat) * np.pi / 360) ** 2 
                                           + np.cos(start_lat * np.pi / 180) 
                                           * np.cos(lat * np.pi / 180) 
                                           * np.sin((start_lng - lng) * np.pi / 360) ** 2))

iarray = None
journeys = None

def calc_iarray(_) :
    global iarray, journeys
    if data is not None :
        start_time = time.time()

        journeys = data["journey_id"].unique()
        iarray = np.zeros((len(TPs.index), len(journeys)), dtype = float)

        iarray_progress = widgets.IntProgress(
            value = 0,
            min = 0,
            max = len(journeys) - 1,
            step = 1,
            description = "Calculating",
            bar_style = "success",
            orientation = "horizontal",
            layout = Layout(width = "40%"),
            style = {"description_width":"initial"}
        )

        display(widgets.HBox([iarray_progress]))

        for i in range(len(journeys)) :
            journey = data.where(data["journey_id"] == journeys[i]).dropna(how = "all")

            for j in range(len(TPs.index)) :
                if int(TPs["guaranteed"].iloc[j]) == -1 :
                    if TPs["TP"].iloc[j] in journey["name#reg"].values :
                        journey_subset = journey.where(journey["name#reg"] == TPs["TP"].iloc[j]).dropna(how = "all").iloc[0]
                        if float(journey_subset["min_dist"]) >= 5 :
                            iarray[j, i] = 5
                        else :
                            iarray[j, i] = float(journey_subset["min_dist"])
                    else :
                        if float(journey["max_m3"].iloc[0]) <= float(TPs["capacity"].iloc[j]) :
                            if float(journey["start_end"].iloc[0]) <= float(TPs["maxkm"].iloc[j]) :
                                if int(journey["number_of_men"].iloc[0]) == int(TPs["1man"].iloc[j]) or int(journey["number_of_men"].iloc[0]) == int(TPs["2man"].iloc[j]) :
                                    if int(journey["tail_lift"].iloc[0]) <= int(TPs["TL"].iloc[j]) :
                                        distance = distance_funct(float(journey["start_lat"].iloc[0]), float(journey["start_lng"].iloc[0]), float(TPs["lat"].iloc[j]), float(TPs["lng"].iloc[j]))
                                        if distance <= (float(TPs["maxkm"].iloc[j]) if float(TPs["maxkm"].iloc[j]) < 2000 else float(TPs["maxkm"].iloc[j]) - 2000) :
                                            iarray[j, i] = distance
                elif TPs["TP"].iloc[j] in journey["name#reg"].values :
                    iarray[j, i] = float(journey.where(journey["name#reg"] == TPs["TP"].iloc[j]).dropna(how = "all")["min_dist"].iloc[0])
            iarray_progress.value = i

        print("Took {0:.3f} secs to run".format(time.time() - start_time))

    else :
        print("No journeys available")
        
iarray_button.on_click(calc_iarray)

display(iarray_button)

Button(description='Calculate IArray', layout=Layout(width='150px'), style=ButtonStyle())

Took 2.190 secs to run


In [9]:
if data is not None :
    journey_menu = widgets.Dropdown(
        options = np.insert(np.sort(journeys), 0, "Select journey"),
        value = "Select journey",
        description = "Journey:")
else :
    journey_menu = widgets.Dropdown(
        options = ["Select journey"],
        value = "Select journey",
        description = "Journey:")

journey_button = widgets.Button(description = "Find", layout = Layout(width = "80px"))
out = widgets.Output()

def journey_display(_) :
    with out :
        clear_output()
        if journey_menu.value != "Select journey" and journey_menu.value != None:
            print("Journey {0}\n".format(journey_menu.value))
            for TP, iarray_val in list(zip(TPs["TP"], iarray.transpose()[np.argwhere(journeys == journey_menu.value)][0][0])) :
                print("{0:20} -> {1}".format(TP, iarray_val))

journey_button.on_click(journey_display)

display(widgets.VBox([widgets.HBox([journey_menu, journey_button]), out]))

In [13]:
journey_menu = widgets.Dropdown(
    options = np.insert(np.sort(TPs["TP"]), 0, "Select TP"),
    value = "Select TP",
    description = "TP:")

journey_button = widgets.Button(description = "Find", layout = Layout(width = "80px"))
out = widgets.Output()

def journey_display(_) :
    with out :
        clear_output()
        if journey_menu.value != "Select TP" and journey_menu.value != None:
            print("TP {0}\n".format(journey_menu.value))
            for journey, iarray_val in list(zip(journeys, iarray[np.argwhere(np.array(TPs["TP"]) == journey_menu.value)][0][0])) :
                print("{0:20} -> {1}".format(journey, iarray_val))

journey_button.on_click(journey_display)

display(widgets.VBox([widgets.HBox([journey_menu, journey_button]), out]))

# Done (Need to add matcher capability now)